In [ ]:
%%HTML
<style type="text/css">

div.h2 {
    background-color: steelblue; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 20px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}
div.h3 {
    color: steelblue; 
    font-size: 14px; 
    margin-top: 20px; 
    margin-bottom:4px;
}
div.h4 {
    font-size: 15px; 
    margin-top: 20px; 
    margin-bottom: 8px;
}
span.note {
    font-size: 5; 
    color: gray; 
    font-style: italic;
}
span.captiona {
    font-size: 5; 
    color: dimgray; 
    font-style: italic;
    margin-left: 130px;
    vertical-align: top;
}
hr {
    display: block; 
    color: gray
    height: 1px; 
    border: 0; 
    border-top: 1px solid;
}
hr.light {
    display: block; 
    color: lightgray
    height: 1px; 
    border: 0; 
    border-top: 1px solid;
}
table.dataframe th 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
}
table.dataframe td 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 14px;
    text-align: center;
} 
table.rules th 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 14px;
}
table.rules td 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 13px;
    text-align: center;
} 
table.rules tr.best
{
    color: green;
}

</style>

<div class=h1> Increasing the amount and diversity of data. </div>

Deep learning techniques have found great success when it comes to computer vision tasks namely image recognition, image segmentation, object detection, etc. Such deep learning techniques are heavily dependent on big data to avoid overfitting. So what do you do when you have limited data? Your go for Data Augmentation. [Data Augmentation techniques that enhance the size and quality of training datasets such that better Deep Learning models can be built using them](https://link.springer.com/content/pdf/10.1186%2Fs40537-019-0197-0.pdf). In this notebook, we shall look at some of the common image augmentation techniques and some of the popular libraries which help to achieve these augmentations. 

<div class=h2>Types of Augmentations </div> 
 
There are broadly two kinds of Augmentations

![](https://imgur.com/vz7gkdD.png)


<div class=h2>Commonly used Augmentation Techniques </div> 
 
Some of the commonly used Image data augmentations techniques are:
* <div class=h3>Flipping  </div> 
This means flipping the image horizontally or vertically
* <div class=h3>Rotation  </div>
This means to rotate the image by a given angle in the clockwise or anticlockwise direction
* <div class=h3>Cropping  </div>
During cropping, a section of the image is sampled randomly
* <div class=h3>Brightness  </div>
Increase or decrease the brightness of the image
* <div class=h3>Scaling </div>Scaling
Images can be scaled outward or inward. When scaled outward, the image size increases while the image size decreases when scaled inwards.
* <div class=h3>Noise Addition </div>
We can also add gaussian noise to the existing images.

Before delving into the data augmentation packages let us understand some points about an image.For that we will need to import the necessary libraries and the image dataset

### Importing necessary libraries and the dataset

In [ ]:

import os
import glob

import numpy as np
import scipy as sp
import pandas as pd

# skimage
from skimage.io import imshow, imread, imsave
from skimage.transform import rotate, AffineTransform, warp,rescale, resize, downscale_local_mean
from skimage import color,data
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

#OpenCV-Python
import cv2

# imgaug
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

# Albumentations
import albumentations as A

# Augmentor
!pip install augmentor
import Augmentor 

# Keras
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img 

# SOLT
!pip install solt
import solt
import solt.transforms as slt

#visualisation
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
from IPython.display import HTML, Image

#source: https://www.kaggle.com/jpmiller/nfl-punt-analytics/edit
# set additional display options for report
pd.set_option("display.max_columns", 100)
th_props = [('font-size', '13px'), ('background-color', 'white'), 
            ('color', '#666666')]
td_props = [('font-size', '15px'), ('background-color', 'white')]
styles = [dict(selector="td", props=td_props), dict(selector="th", 
            props=th_props)]

#warnings
import warnings
warnings.filterwarnings("ignore")


#Helper function to display the images in a grid
# Source: https://stackoverflow.com/questions/42040747/more-idiomatic-way-to-display-images-in-a-grid-with-numpy which was pointed by
# this excellent article: https://towardsdatascience.com/data-augmentation-for-deep-learning-4fe21d1a4eb9
def gallery(array, ncols=3):
    '''
    Function to arange images into a grid.
    INPUT:
        array - numpy array containing images
        ncols - number of columns in resulting imahe grid
    OUTPUT:
        result - reshaped array into a grid with given number of columns
    '''
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result



In [ ]:
# Defining data path
Image_Data_Path = "../input/plant-pathology-2020-fgvc7/images/"

train_data = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")
test_data = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

# Loading the training images #refer: https://www.kaggle.com/tarunpaparaju/plant-pathology-2020-eda-models
def load_image(image_id):
    file_path = image_id + ".jpg"
    image = imread(Image_Data_Path + file_path)
    return image

train_images = train_data["image_id"][:50].apply(load_image)

## A look at some of the images in the training set

In [ ]:

# image titles
#image_titles = ['Frame1', 'Frame2', 'Frame3']

# plotting multiple images using subplots
fig,ax = plt.subplots(nrows=2,ncols=3,figsize=(30,16))
for col in range(3):
    for row in range(2):
        ax[row,col].imshow(train_images.loc[train_images.index[row*3+col]])
        #ax[row,col].set_title(image_titles[i])    
        ax[row,col].set_xticks([])
        ax[row,col].set_yticks([])

<div class=h2>Images as Arrays </div>
An image is nothing but a standard Numpy array containing pixels of data points. You can think of pixels to be tiny blocks of information arranged in the form of a 2 D grid, and the depth of a pixel refers to the colour information present in it

In [ ]:
image = train_images[15]
imshow(image)
print(image.shape)

**Coloured images** are represented as a combination of Red, Blue, and Green, and all the other colours can be achieved by mixing these primary colours in the correct proportions.

In [ ]:
# red filter [R,G,B]
red_filter = [1,0,0]
# blue filter
blue_filter = [0,0,1]
# green filter
green_filter = [0,1,0]


# matplotlib code to display
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(30,16))
ax[0].imshow(image*red_filter)
ax[0].set_title("Red Filter",fontweight="bold", size=30)
ax[1].imshow(image*blue_filter)
ax[1].set_title("BLue Filter",fontweight="bold", size=30)
ax[2].imshow(image*green_filter)
ax[2].set_title("Green Filter",fontweight="bold", size=30);

A **grayscale image** consists of 8 bits per pixel. This means it can have 256 different shades where 0 pixels will represent black colour while 255 denotes white. For example, the image below shows a grayscale image represented in the form of an array. A grayscale image has only 1 channel where the channel represents dimension.

In [ ]:
# import color sub-module
from skimage import color

# converting image to grayscale
grayscale_image = color.rgb2gray(image)
grayscale_image.shape
imshow(grayscale_image)

<div class=h1>Popular Image Augmentation packages </div> 

There are a lot of image augmentations packages. However, we shall cover some of the popular ones like:

* <div class=h3> skimage </div> 
* <div class=h3> opencv </div> 
* <div class=h3> imgaug </div> 
* <div class=h3> Albumentations </div> 
* <div class=h3> Augmentor </div> 
* <div class=h3> Keras(ImageDataGenerator class) </div> 
* <div class=h3> SOLT </div> 


# 1. Data Augmentation using skimage 
  
[scikit-image](https://scikit-image.org/) is an open source Python package that works with numpy arrays. It implements algorithms and utilities for use in research, education and industry applications. It is a fairly simple and straightforward library even for those who are new to Python’s ecosystem. This code is of high-quality and peer-reviewed, written by an active community of volunteers.

<div class=h3> 1.1 Flipping with skimage </div> 

In [ ]:
#Horizontally flipped
hflipped_image= np.fliplr(image) #fliplr reverse the order of columns of pixels in matrix

#Vertically flipped
vflipped_image= np.flipud(image) #flipud reverse the order of rows of pixels in matrix

fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=30)
ax[1].imshow(hflipped_image)
ax[1].set_title("Horizontally flipped", size=30)
ax[2].imshow(vflipped_image)
ax[2].set_title("Vertically flipped", size=30);


<div class=h3> 1.2 Rotation with skimage </div> 

In [ ]:
# clockwise rotation
rot_clockwise_image = rotate(image, angle=45) 
# Anticlockwise rotation
rot_anticlockwise_image = rotate(image, angle=-45)



In [ ]:

fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=30)
ax[1].imshow(rot_clockwise_image)
ax[1].set_title("+45 degree Rotation", size=30)
ax[2].imshow(rot_anticlockwise_image)
ax[2].set_title("-45 degree rotation", size=30);

<div class=h3> 1.3 Cropping with skimage </div> 

In [ ]:
# source: https://www.kaggle.com/safavieh/image-augmentation-using-skimage
import random
import pylab as pl 
def randRange(a, b):
    '''
    a utility function to generate random float values in desired range
    '''
    return pl.rand() * (b - a) + a
def randomCrop(im):
    '''
    croping the image in the center from a random margin from the borders
    '''
    margin = 1/3.5
    start = [int(randRange(0, im.shape[0] * margin)),
             int(randRange(0, im.shape[1] * margin))]
    end = [int(randRange(im.shape[0] * (1-margin), im.shape[0])), 
           int(randRange(im.shape[1] * (1-margin), im.shape[1]))]
    cropped_image = (im[start[0]:end[0], start[1]:end[1]])
    return cropped_image


In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,12))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(randomCrop(image))
ax[1].set_title("Cropped", size=20)

<div class=h3> 1.4 Brightness Manipulation </div> 

In [ ]:

image_bright = adjust_gamma(image, gamma=0.5,gain=1)
image_dark = adjust_gamma(image, gamma=2,gain=1)

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(20,12))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(image_bright)
ax[1].set_title("Brightened Image", size=20)
ax[2].imshow(image_dark)
ax[2].set_title("Darkened Image", size=20)

<div class=h3> 1.5 Scaling with skimage </div> 

In [ ]:

image_resized = resize(image, (image.shape[0] // 2, image.shape[1] // 2),
                       anti_aliasing=True)
#image_downscaled = downscale_local_mean(image, (4, 3))


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(image_resized)
ax[1].set_title("Resized image",size=20)


<div class=h3> 1.6 Noise Addition with skimage </div> 

In [ ]:
noisy_image= random_noise(image)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(noisy_image)
ax[1].set_title("Image after adding noise",size=20)

# 2. Data Augmentation using OpenCV-Python

![](https://opencv-python-tutroals.readthedocs.io/en/latest/_static/opencv-logo-white.png)

[OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html) essentially stands for Open Source Computer Vision Library. Although it is written in optimized C/C++, it has interfaces for Python and Java along with C++. 
OpenCV-Python is the python API for OpenCV. You can think of it as a python wrapper around the C++ implementation of OpenCV. OpenCV-Python is not only fast (since the background consists of code written in C/C++) but is also easy to code and deploy(due to the Python wrapper in foreground). This makes it a great choice to perform computationally intensive programs.


In [ ]:
# selecting a sample image
image13 = train_images[13]
imshow(image13)
print(image13.shape)
plt.axis('off')


<div class=h3> 2.1 Flipping with opencv </div> 

The image is flipped according to the value of flipCode as follows:

* flipcode = 0: flip vertically
* flipcode > 0: flip horizontally
* flipcode < 0: flip vertically and horizontally

In [ ]:
#vertical flip
img_flip_ud = cv2.flip(image13, 0)
plt.imshow(img_flip_ud)

#horizontal flip
img_flip_lr = cv2.flip(image13, 1)
plt.imshow(img_flip_lr)


In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,12))
ax[0].imshow(img_flip_ud)
ax[0].set_title("vertical flip", size=20)
ax[1].imshow(img_flip_lr)
ax[1].set_title("horizontal flip", size=20)


<div class=h3> 2.2 Rotation with opencv </div> 

The OpenCV function that rotates the image is `cv2.rotate()`.The following three constants can be specified in rotateCode.

```
cv2.ROTATE_90_CLOCKWISE
cv2.ROTATE_90_COUNTERCLOCKWISE
cv2.ROTATE_180
```


In [ ]:
img_rotate_90_clockwise = cv2.rotate(image13, cv2.ROTATE_90_CLOCKWISE)
img_rotate_90_counterclockwise = cv2.rotate(image13, cv2.ROTATE_90_COUNTERCLOCKWISE)
img_rotate_180 = cv2.rotate(image13, cv2.ROTATE_180)


In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(20,12))
ax[0].imshow(img_rotate_90_clockwise)
ax[0].set_title("90 degrees clockwise", size=20)
ax[1].imshow(img_rotate_90_counterclockwise)
ax[1].set_title("90 degrees anticlockwise", size=20)
ax[2].imshow(img_rotate_180)
ax[2].set_title("180 degree rotation", size=20)

<div class=h3> 2.3 Scaling with opencv </div> 

Scaling is just resizing of the image

In [ ]:
#RESIZE
def resize_image(image,w,h):
    resized_image = image=cv2.resize(image,(w,h))
    return resized_image

imshow(resize_image(image13, 500,500))

<div class=h3> 2.4 Brightness Manipulation with opencv </div> 


In [ ]:
def add_light(image, gamma):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    image=cv2.LUT(image, table)
    return image

imshow(add_light(image13,2))

<div class=h3> 2.5 Cropping with opencv </div> 


In [ ]:
#crop
def crop_image(image,y1,y2,x1,x2):
    image=image[y1:y2,x1:x2]
    return image
imshow(crop_image(image13,200,800,250,1500))#(y1,y2,x1,x2)(bottom,top,left,right)


<div class=h3> 2.6 Gaussian Blur with opencv </div> 

The Gaussian filter is a low-pass filter that removes the high-frequency components are reduced.

In [ ]:
def gaussian_blur(image,blur):
    image = cv2.GaussianBlur(image,(5,5),blur)
    return image

imshow(gaussian_blur(image13,0))

# 3. Data Augmentation using imgaug  
  

[imgaug](https://imgaug.readthedocs.io/en/latest/) is a library for image augmentation in machine learning experiments. It supports a wide range of augmentation techniques, allows to easily combine these and to execute them in random order or on multiple CPU cores, has a simple yet powerful stochastic interface and can not only augment images, but also keypoints/landmarks, bounding boxes, heatmaps and segmentation maps.
![](https://cdn-images-1.medium.com/max/800/1*QT3A5EZIp1EXSVIiB4SlCA.png)



In [ ]:
# selecting a sample image
image2 = train_images[25]
imshow(image2)
print(image2.shape)
plt.axis('off')

<div class=h3> 2.1 Flipping with imgaug </div> 

In [ ]:

#Horizontally flipped
hflip= iaa.Fliplr(p=1.0)
hflipped_image2= hflip.augment_image(image2)

#Vertically flipped
vflip= iaa.Flipud(p=1.0) 
vflipped_image2= vflip.augment_image(image2)


In [ ]:
image=image2
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=30)
ax[1].imshow(hflipped_image2)
ax[1].set_title("Horizontally flipped", size=30)
ax[2].imshow(vflipped_image2)
ax[2].set_title("Vertically flipped", size=30);

<div class=h3> 2.2 Rotation with imgaug </div> 

In [ ]:
# clockwise rotation
rot = iaa.Affine(rotate=(-25,25))
rot_clockwise_image2 = rot.augment_image(image2)


In [ ]:
image=image2
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(30,16))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=30)
ax[1].imshow(rot_clockwise_image2)
ax[1].set_title("Rotated Image", size=30)


<div class=h3> 2.3 Cropping with imgaug </div> 

In [ ]:

image=image2
crop = iaa.Crop(percent=(0, 0.2)) # crop image
corp_image=crop.augment_image(image)


In [ ]:

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,12))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(randomCrop(corp_image))
ax[1].set_title("Cropped", size=20)

<div class=h3> 2.4 Brightness Manipulation with imgaug </div> 

In [ ]:
# 4. Brightness

image = image2
# bright
#contrast1=iaa.GammaContrast(gamma=0.5)
#brightened_image = contrast1.augment_image(image)

#dark
#contrast2=iaa.GammaContrast(gamma=2)
#darkened_image = contrast2.augment_image(image)

#Somehow, this line of code gave me an error in the Kaggle notebook but worked fine in other IDEs. 
#So I have inserted the desired result.ULet me know if the above code works for you?

In [ ]:

#fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(20,12))
#ax[0].imshow(image)
#ax[0].set_title("Original Image", size=20)
#ax[1].imshow(brightened_image)
#ax[1].set_title("Brightened Image", size=20)
#ax[2].imshow(darkened_image)
#ax[2].set_title("darkened_image", size=20)



![](https://imgur.com/cwg7Oiy.png)

<div class=h3> 2.5 Scaling with imgaug </div> 

In [ ]:
image= image2
scale_im=iaa.Affine(scale={"x": (1.5, 1.0), "y": (0.5, 1.0)})
scale_image =scale_im.augment_image(image)

In [ ]:

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,12))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(scale_image)
ax[1].set_title("Scaled", size=20)

<div class=h3> 2.6 Noise addition with imgaug </div> 

In [ ]:
image= image2
gaussian_noise=iaa.AdditiveGaussianNoise(15,20)
noise_image=gaussian_noise.augment_image(image)


In [ ]:

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,12))
ax[0].imshow(image)
ax[0].set_title("Original Image", size=20)
ax[1].imshow(noise_image)
ax[1].set_title("Gaussian Noise added", size=20)

<div class=h3>2.7 Augmentation pipeline </div> 

The imgaug library provides a very useful feature called **Augmentation pipeline**. Such a pipeline is a sequence of steps that can be applied in a fixed or random order. This also gives the flexibility to apply certain transformations to a few images and other transformations to other images. In the following example, we are applying the flip, sharpen,crop etc transformations on some of the images. The blur and affline transformations will be applied sometimes and all these transformations will be applied in random order.

In [ ]:
# Defining a pipeline.
# The example has been taken from the documentation
aug_pipeline = iaa.Sequential([
    iaa.SomeOf((0,3),[
        iaa.Fliplr(1.0), # horizontally flip
        iaa.Flipud(1.0),# Vertical flip
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
        iaa.Crop(percent=(0, 0.4)),
        iaa.Sometimes(0.5, iaa.Affine(rotate=5)),
        iaa.Sometimes( 0.5,iaa.GaussianBlur(sigma=(0, 0.5))),
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    ])
], 
random_order=True # apply the augmentations in random order
)

# apply augmentation pipeline to sample image
images_aug = np.array([aug_pipeline.augment_image(image2) for _ in range(16)])

# visualize the augmented images
plt.figure(figsize=(30,10))
plt.axis('off')
plt.imshow(gallery(images_aug, ncols = 4))
plt.title('Augmentation  examples')


# 4. Data Augmentation using Albumentations 

![](https://albumentations.readthedocs.io/en/latest/_static/logo.png)

[Albumentations](https://albumentations.readthedocs.io/en/latest/index.html#) is a fast image augmentation library and easy to use wrapper around other libraries.It is based on numpy, OpenCV, imgaug picking the best from each of them.It is written by Kagglers and was used to get top results in many DL competitions at Kaggle, topcoder, CVPR, MICCAI. Read more about it here: https://www.mdpi.com/2078-2489/11/2/125

In [ ]:
# initialize augmentations
horizontal_flip = A.HorizontalFlip(p=1)
rotate = A.ShiftScaleRotate(p=1)
gaus_noise = A.GaussNoise() # gaussian noise
bright_contrast = A.RandomBrightnessContrast(p=1) # random brightness and contrast
gamma = A.RandomGamma(p=1) # random gamma
blur = A.Blur()

# apply augmentations to images
img_flip = horizontal_flip(image = image2)
img_gaus = gaus_noise(image = image2)
img_rotate = rotate(image = image2)
img_bc = bright_contrast(image = image2)
img_gamma = gamma(image = image2)
img_blur = blur(image = image2)

# access the augmented image by 'image' key
img_list = [img_flip['image'],img_gaus['image'], img_rotate['image'], img_bc['image'], img_gamma['image'], img_blur['image']]

# visualize the augmented images
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(gallery(np.array(img_list), ncols = 3))
plt.title('Augmentation examples')

# 5. Data Augmentation using Augmentor 

![](https://augmentor.readthedocs.io/en/master/_static/logo.png)


In [ ]:
# Passing the path of the image directory 
p = Augmentor.Pipeline(source_directory="/kaggle/input/plant-pathology-2020-fgvc7/images",
                      output_directory="/kaggle/output")
  
# Defining augmentation parameters and generating 10 samples 
p.flip_left_right(probability=0.4) 
p.flip_top_bottom(probability=0.8)
p.rotate(probability=0.5, max_left_rotation=5, max_right_rotation=10)
p.skew(0.4, 0.5) 
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5) 
p.sample(10)

# 6. Keras Image Data Generator  

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRgbD4KXC9PBSYSLHojvt-qcu99NCfy4AcN3eEGFM1YTmLIAJFo&usqp=CAU)
The Keras library has a built in class created just for the purpose of adding transformations to images.This class is called **ImageDataGenerator** and it generates batches of tensor image data with real-time data augmentations. 

In [ ]:
# selecting a sample image
image5 = train_images[15]
imshow(image5)
print(image5.shape)
plt.axis('off')

Let's create a directory where the transformed images will be stored. The directory will be called keras_augmentation and its path is as follows:


In [ ]:
# To delete any previously created directory
#import shutil
#shutil.rmtree("../output/keras_augmentations")


# Creating a new directory for placing augmented images
import os
os.mkdir("../output/keras_augmentations")



In [ ]:
# Augmentation process
datagen = ImageDataGenerator( 
        rotation_range = 40, 
        shear_range = 0.2, 
        zoom_range = 0.2, 
        horizontal_flip = True, 
        brightness_range = (0.5, 1.5)) 

img_arr = img_to_array(image5)
img_arr = img_arr.reshape((1,) + img_arr.shape)

i = 0
for batch in datagen.flow(
    img_arr,
    batch_size=1,
    save_to_dir='../output/keras_augmentations',
    save_prefix='Augmented_image',
    save_format='jpeg'):
    i += 1
    if i > 20: # create 20 augmented images
        break  # otherwise the generator would loop indefinitely

These are just a few of the options available (for more, see the [documentation](https://keras.io/preprocessing/image/)). Let's see what do these transformation mean- Source: [Keras Blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

* **rotation_range** is a value in degrees (0-180), a range within which to randomly rotate pictures
* shear_range is for randomly applying [shearing transformations](https://en.wikipedia.org/wiki/Shear_mapping)
* zoom_range is for randomly zooming inside pictures
* horizontal_flip is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).
* fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

Let's look at the images which have been created in our directory

In [ ]:
images = os.listdir("../output/keras_augmentations/")
images

In [ ]:
# Let's look at the augmented images
aug_images = []
for img_path in glob.glob("../output/keras_augmentations/*.jpeg"):
    aug_images.append(mpimg.imread(img_path))

plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(aug_images):
    plt.subplot(len(aug_images) / columns + 1, columns, i + 1)
    plt.imshow(image)



In [ ]:
# selecting a sample image
image5 = train_images[25]
imshow(image5)
print(image5.shape)
plt.axis('off')

# 7. SOLT : Streaming over lightweight data transformations 

![](https://github.com/MIPT-Oulu/solt/raw/master/doc/source/_static/logo.png)
[SOLT](https://github.com/MIPT-Oulu/solt) is a fast data augmentation library, supporting arbitrary amount of images, segmentation masks, keypoints and data labels. It has OpenCV in its back-end, thus it works very fast.

In [ ]:
h,w,c = image5.shape
img = image5[:w]

In [ ]:
stream = solt.Stream([
    slt.Rotate(angle_range=(-90, 90), p=1, padding='r'),
    slt.Flip(axis=1, p=0.5),
    slt.Flip(axis=0, p=0.5),
    slt.Shear(range_x=0.3, range_y=0.8, p=0.5, padding='r'),
    slt.Scale(range_x=(0.8, 1.3), padding='r', range_y=(0.8, 1.3), same=False, p=0.5),
    slt.Pad((w, h), 'r'),
    slt.Crop((w, w), 'r'),
    slt.CvtColor('rgb2gs', keep_dim=True, p=0.2),
    slt.HSV((0, 10), (0, 10), (0, 10)),
    slt.Blur(k_size=7, blur_type='m'),
    solt.SelectiveStream([
        slt.CutOut(40, p=1),
        slt.CutOut(50, p=1),
        slt.CutOut(10, p=1),
        solt.Stream(),
        solt.Stream(),
    ], n=3),
], ignore_fast_mode=True)

In [ ]:
fig = plt.figure(figsize=(16,16))
n_augs = 6


random.seed(42)
for i in range(n_augs):
    img_aug = stream({'image': img}, return_torch=False, ).data[0].squeeze()

    ax = fig.add_subplot(1,n_augs,i+1)
    if i == 0:
        ax.imshow(img)
    else:
        ax.imshow(img_aug)
    ax.set_xticks([])
    ax.set_yticks([])

plt.show()


### References and Some great resources :

- [Data Augmentation | How to use Deep Learning when you have Limited Data — Part 2](https://nanonets.com/blog/data-augmentation-how-to-use-deep-learning-when-you-have-limited-data-part-2/)
- [Data Augmentation for Deep Learning](https://towardsdatascience.com/data-augmentation-for-deep-learning-4fe21d1a4eb9)


